In [4]:
import sys
if sys.path[0].endswith('/src'):
    sys.path.insert(0, sys.path[0].removesuffix('/src'))
print('Path:', sys.path)

import yfinance as yf
import pandas as pd
import datetime
import pytz
import os

from calcEMA import *

Path: ['c:\\Users\\mlimag\\des\\analise_ativos_mercado_financeiro\\src', 'C:\\Users\\mlimag\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\mlimag\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\mlimag\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\mlimag\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\mlimag\\des\\analise_ativos_mercado_financeiro\\.env', '', 'c:\\Users\\mlimag\\des\\analise_ativos_mercado_financeiro\\.env\\Lib\\site-packages', 'c:\\Users\\mlimag\\des\\analise_ativos_mercado_financeiro\\.env\\Lib\\site-packages\\win32', 'c:\\Users\\mlimag\\des\\analise_ativos_mercado_financeiro\\.env\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\mlimag\\des\\analise_ativos_mercado_financeiro\\.env\\Lib\\site-packages\\Pythonwin']


In [ ]:
def process(load_cache = True):
    data_file = sys.path[0] + '/src/data/ibov.csv'
    print('Carregando Dataset...')
    dataset = load_dataset(load_cache, data_file)     
    print('Iniciando Calculo RSI e EMAs...')

    emas_dataset = pd.DataFrame()
    for symbol in get_tickers():
        emas_dataset = pd.concat([emas_dataset, run_calc_emas(dataset[dataset['symbol'] == symbol], 'adj_close')])
    
    print('Lista ordenada por *Ações com Desconto*:', emas_dataset.index.max())
    print_descontados(emas_dataset)


    emas_dataset.to_csv(
            data_file,
            sep=';',
            )
    return emas_dataset

In [ ]:
def load_dataset(load_cache = True, data_file = './src/data/ibov.json') -> pd.DataFrame:
    symbols = get_tickers()    
    if ( load_cache and os.path.exists(data_file)):
        dataset = pd.read_json(data_file, orient='records', date_unit='s')
        dataset.index = pd.to_datetime(dataset['date_time'])
        dataset['date_import'] = pd.to_datetime(dataset['date_import'])
        dataset.index.name = 'date'
    else:
        data = download_data('2013-01-01', symbols)
        dataset = convert_downloaded_data(data)

    print(dataset.info())
    return dataset

In [ ]:
def get_tickers() -> list:
    filename = sys.path[0] + '/src/data/tickers_list_to_analisys.csv'
    print('Tickers List File:', filename)
    tickers = pd.read_csv(filename)
    tickers['symbol'] += '.SA'
    return list(tickers['symbol'])

In [ ]:
def convert_downloaded_data(tickers_history: pd.DataFrame) -> pd.DataFrame:
    symbols = []
    for symbol, _ in tickers_history.columns:
        symbols.append(symbol)
    # Remove duplicates
    symbols = list(set(symbols))

    new_df = pd.DataFrame()
    for s in symbols:
        aux = tickers_history[s].copy()
        aux['symbol'] = s
        new_df = pd.concat([new_df, aux], axis=0)
    
    new_df.dropna(how='any', axis=0, inplace=True) 
    new_df.rename(columns={'Adj Close': 'adj_close', 'Close': 'close', 'High': 'high',
                  'Low': 'low', 'Open': 'open', 'Close': 'close', 'Volume': 'volume'}, inplace=True)
    new_df.index.name = 'date'
    new_df['date_time'] = pd.to_datetime(new_df.index)
    new_df['date_import'] = pd.to_datetime(datetime.datetime.now(tz=pytz.UTC))
    return new_df

In [ ]:
def download_data(start_date='', tickers=[]) -> pd.DataFrame:
    if start_date == '':
        year = datetime.datetime.today().year
        start_date = str(year) + '-01-01'

    print('Baixando dados [start_date]: ' + start_date)
    print('Symbols: ', tickers)
    data = yf.download(tickers, start=start_date,
                       threads=20, group_by='ticker')
    return data

In [ ]:
def print_descontados(df: pd.DataFrame):
    filter = df[df.index == df.index.max()]
    print(filter.sort_values(by='ema_200p_diff', ascending=True))

## Validando Código

In [ ]:
dataset = process()

In [ ]:
dataset.groupby(by='symbol').count()

In [ ]:
dataset.to_csv('teste.csv', sep=';', index=False)

In [ ]:
dataset.isna()

# Testando RSI

In [ ]:
def cRsi(df: pd.DataFrame, close_price='close', window=14):
    '''
    # Create two copies of the Closing price Series
    change_up = df.copy()
    change_down = df.copy()

    # Calculate the rolling average of average up and average down
    avg_up = change_up[close_price].rolling(14).mean()
    avg_down = change_down[close_price].rolling(14).mean().abs()

    rsi = 100 * avg_up / (avg_up + avg_down)
    _df = df.copy()
    _df['rsi'] = rsi
    return _df
    '''
    aux = df.copy()
    try:
        aux['change'] = aux[close_price].diff()
        aux['gain'] = aux.change.mask(aux.change < 0, 0.0)
        aux['loss'] = -aux.change.mask(aux.change > 0, -0.0)
        aux['avg_gain'] = rma(aux.gain.to_numpy(), window)
        aux['avg_loss'] = rma(aux.loss.to_numpy(), window)

        aux['rs'] = aux.avg_gain / aux.avg_loss
        aux['rsi'] = 100 - (100 / (1 + aux.rs))


    except Exception as error:
        print('Erro no calculo do RSI> ', df['symbol'], ' - Data: ', df['date_time'])
        print(error)
        aux['rsi'] = 0.0
    finally:
        aux.drop(columns=['change', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs'], inplace=True, errors='ignore')
    return aux

In [ ]:
data2 = load_dataset(load_cache=True)
data2

In [ ]:
data22 = pd.DataFrame()
print(get_tickers())
for symbol in get_tickers():  
  print(symbol, ':', data2[data2['symbol'] == symbol]['symbol'].count())
  if data2[data2['symbol'] == symbol]['symbol'].count() > 14:
    rsi_df = cRsi(data2[data2['symbol'] == symbol])
    data22 = pd.concat([data22, rsi_df])
    print(rsi_df.tail(1)[['symbol', 'rsi']])
data22

In [ ]:
__df = data2[data2['symbol'] == symbol]
print(__df['symbol'].count())
print(__df.tail(1))
rsi_df = cRsi(__df)
rsi_df

In [ ]:
ultimo_dia = data22.index.max()

# data22[data22.index == ultimo_dia]
data22.info()

In [ ]:
data3 = yf.download(get_tickers(), start='2013-01-01',
                       threads=20, group_by='ticker')

In [ ]:
data3

In [ ]:
new_data3 = convert_downloaded_data(data3)
new_data3

In [ ]:
new_data3.dropna(how='any', axis=0, inplace=True)

new_data3

In [ ]:
new_data3.groupby(by='symbol').count()

## Lendo arquivo de cache e atualizando com dados faltantes

In [18]:
dados = pd.read_csv('.\data\ibov.csv', sep=';', index_col='date', parse_dates=True)
dados['date_time'] = pd.to_datetime(dados['date_time'])
dados['date_import'] = pd.to_datetime(dados['date_import'])

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 200616 entries, 2020-11-13 to 2023-06-26
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   open           200616 non-null  float64            
 1   high           200616 non-null  float64            
 2   low            200616 non-null  float64            
 3   close          200616 non-null  float64            
 4   adj_close      200616 non-null  float64            
 5   volume         200616 non-null  float64            
 6   symbol         200616 non-null  object             
 7   date_time      200616 non-null  datetime64[ns]     
 8   date_import    200616 non-null  datetime64[ns, UTC]
 9   ema_200p       183104 non-null  float64            
 10  ema_200p_diff  183104 non-null  float64            
 11  rsi            199243 non-null  float64            
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(9), object(1)
me

In [26]:
max_date = dados.index.max()
print(max_date.strftime('%Y-%m-%d'))
print(datetime.datetime.now().strftime('%Y-%m-%d'))
max_date > datetime.datetime.now()

2023-06-26
2023-06-26


False